In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'gabor'
CHANNEL = 'red'
PARAM_CSV = 'gabor_new.csv' # only use for Gabor

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_red_gabor_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))
param_df = pd.read_csv(os.path.join(ROOT_DIR, "gabor", PARAM_CSV))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters
elif 'gabor' in TRANSFORM:
    GROUPS = param_df['index']

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,46126.74,1.37,-0.23,0.49,0.13,-0.16,-0.14,-2.30,0.73,-0.49,...,-0.19,0.37,-0.03,0.84,-0.12,-0.49,0.22,0.05,0.03,0.05
1,1.37,18141.28,0.66,-0.48,-0.06,-0.10,-0.01,1.51,-0.26,-0.39,...,0.00,-0.00,0.01,0.97,-0.25,-0.12,-0.30,0.07,0.05,0.08
2,-0.23,0.66,7360.63,-0.13,-0.00,0.08,0.04,0.10,0.53,-0.16,...,0.05,-0.04,0.03,-0.69,0.58,-0.02,-0.35,-0.01,0.09,-0.02
3,0.49,-0.48,-0.13,3073.35,-0.05,0.03,-0.02,-0.65,-0.24,0.01,...,-0.04,-0.06,0.02,0.35,-0.02,0.13,0.07,0.04,-0.04,-0.00
4,0.13,-0.06,-0.00,-0.05,1221.28,-0.01,0.01,-0.02,0.00,0.24,...,0.00,0.01,0.01,0.02,0.14,-0.10,0.09,0.01,0.03,-0.02
5,-0.16,-0.10,0.08,0.03,-0.01,442.20,0.00,-0.28,0.10,-0.03,...,-0.02,-0.01,0.01,-0.16,0.01,0.01,0.04,0.00,0.00,-0.00
6,-0.14,-0.01,0.04,-0.02,0.01,0.00,297.55,0.07,0.00,0.09,...,0.02,-0.00,-0.01,-0.06,-0.02,0.02,-0.03,0.03,-0.01,0.00
7,-2.30,1.51,0.10,-0.65,-0.02,-0.28,0.07,60847.98,-0.13,0.61,...,-0.03,0.16,-0.14,0.47,-1.56,-0.42,-0.60,0.33,0.19,-0.14
8,0.73,-0.26,0.53,-0.24,0.00,0.10,0.00,-0.13,24837.08,0.39,...,-0.12,0.05,0.00,0.67,0.95,-0.79,0.22,0.17,-0.04,-0.04


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

index,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
index,,,,,,,,,,,,,,,,,,,,,
0,1.00000,0.00005,-0.00001,0.00004,0.00002,-0.00004,-0.00004,-0.00004,0.00002,-0.00002,...,-0.00003,0.00008,-0.00001,0.00002,-0.00000,-0.00003,0.00002,0.00001,0.00001,0.00002
1,0.00005,1.00000,0.00006,-0.00006,-0.00001,-0.00004,-0.00000,0.00005,-0.00001,-0.00003,...,0.00000,-0.00000,0.00000,0.00003,-0.00001,-0.00001,-0.00004,0.00001,0.00002,0.00004
2,-0.00001,0.00006,1.00000,-0.00003,-0.00000,0.00004,0.00003,0.00000,0.00004,-0.00002,...,0.00002,-0.00002,0.00003,-0.00004,0.00005,-0.00000,-0.00007,-0.00000,0.00005,-0.00001
3,0.00004,-0.00006,-0.00003,1.00000,-0.00003,0.00002,-0.00002,-0.00005,-0.00003,0.00000,...,-0.00002,-0.00005,0.00003,0.00003,-0.00000,0.00003,0.00002,0.00002,-0.00003,-0.00001
4,0.00002,-0.00001,-0.00000,-0.00003,1.00000,-0.00002,0.00001,-0.00000,0.00000,0.00007,...,0.00000,0.00002,0.00001,0.00000,0.00003,-0.00003,0.00004,0.00001,0.00004,-0.00004
5,-0.00004,-0.00004,0.00004,0.00002,-0.00002,1.00000,0.00000,-0.00005,0.00003,-0.00002,...,-0.00003,-0.00002,0.00003,-0.00003,0.00000,0.00001,0.00004,0.00000,0.00000,-0.00001
6,-0.00004,-0.00000,0.00003,-0.00002,0.00001,0.00000,1.00000,0.00002,0.00000,0.00005,...,0.00004,-0.00001,-0.00002,-0.00001,-0.00001,0.00001,-0.00003,0.00005,-0.00003,0.00001
7,-0.00004,0.00005,0.00000,-0.00005,-0.00000,-0.00005,0.00002,1.00000,-0.00000,0.00002,...,-0.00000,0.00003,-0.00004,0.00001,-0.00004,-0.00002,-0.00004,0.00004,0.00003,-0.00004
8,0.00002,-0.00001,0.00004,-0.00003,0.00000,0.00003,0.00000,-0.00000,1.00000,0.00003,...,-0.00002,0.00001,0.00000,0.00002,0.00004,-0.00006,0.00002,0.00003,-0.00001,-0.00002


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

14.864443469746718

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[8.85356264e+07 6.36630712e+07 5.56701070e+07 5.09699108e+07
 4.75550818e+07 4.34369199e+07 1.46926226e+07 1.02409963e+07
 9.56988634e+06 8.08396909e+06 7.71911949e+06 6.87574009e+06
 2.36313292e+06 1.64212126e+06 1.46806716e+06 1.29443423e+06
 1.21709898e+06 1.07901309e+06 3.92136983e+05 2.75356887e+05
 2.55732920e+05 2.22615775e+05 1.99534790e+05 1.70261535e+05
 6.71715914e+04 4.33393766e+04 3.91793510e+04 3.58134777e+04
 3.44284270e+04 2.66210217e+04 9.54741859e+03 7.45601402e+03
 6.49563522e+03 5.09388758e+03 4.87144450e+03 3.60561320e+03
 2.21512341e+03 1.82440772e+03 1.34770561e+03 1.14771127e+03
 1.03619525e+03 6.86344512e-24]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,-0.036561,-0.006484,-0.002438,-0.001018,-0.000399,-0.000149,-0.000096,0.995284,-0.009778,-0.003342,...,-0.000392,-0.000148,-0.000068,-0.043640,-0.007426,-0.002621,-0.001092,-0.000415,-0.000172,-0.000074
1,-0.083467,-0.006904,-0.002511,-0.001044,-0.000410,-0.000144,-0.000102,0.048037,-0.010840,-0.003453,...,-0.000395,-0.000148,-0.000067,-0.145271,-0.007908,-0.002659,-0.001100,-0.000438,-0.000180,-0.000070
2,-0.231862,-0.007214,-0.002592,-0.001053,-0.000418,-0.000154,-0.000103,0.033304,-0.011598,-0.003559,...,-0.000403,-0.000156,-0.000068,0.954851,-0.008345,-0.002751,-0.001149,-0.000434,-0.000185,-0.000078
3,0.909921,-0.007656,-0.002718,-0.001099,-0.000435,-0.000163,-0.000111,0.028381,-0.012568,-0.003770,...,-0.000430,-0.000155,-0.000073,0.155071,-0.008938,-0.002916,-0.001185,-0.000460,-0.000192,-0.000078
4,0.208919,-0.008976,-0.003112,-0.001270,-0.000506,-0.000186,-0.000119,0.027840,-0.014970,-0.004341,...,-0.000500,-0.000191,-0.000080,0.104512,-0.010442,-0.003322,-0.001367,-0.000525,-0.000218,-0.000087
5,0.254613,-0.023886,-0.008166,-0.003304,-0.001310,-0.000473,-0.000321,0.060574,-0.040864,-0.011346,...,-0.001290,-0.000495,-0.000218,0.170128,-0.027894,-0.008746,-0.003577,-0.001385,-0.000579,-0.000235
6,0.017476,-0.038442,-0.007746,-0.003010,-0.001176,-0.000422,-0.000286,0.011365,0.992813,-0.011465,...,-0.001159,-0.000442,-0.000194,0.016187,-0.058390,-0.008460,-0.003222,-0.001232,-0.000518,-0.000212
7,0.007581,-0.056714,-0.005686,-0.002100,-0.000845,-0.000298,-0.000199,0.005185,0.035741,-0.008777,...,-0.000824,-0.000301,-0.000138,0.007186,-0.353808,-0.006142,-0.002273,-0.000864,-0.000357,-0.000151
8,0.013931,-0.150667,-0.011367,-0.004202,-0.001637,-0.000592,-0.000399,0.009668,0.057859,-0.017902,...,-0.001614,-0.000605,-0.000275,0.013110,0.912787,-0.012494,-0.004530,-0.001732,-0.000714,-0.000295
9,0.002096,-0.627792,-0.002238,-0.000753,-0.000307,-0.000096,-0.000072,0.001483,0.007046,-0.003466,...,-0.000286,-0.000110,-0.000048,0.001983,0.022393,-0.002360,-0.000820,-0.000315,-0.000137,-0.000057


In [13]:
cos_dist = spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine')
cos_dist =  [np.min(row) for row in cos_dist]
cos_dist


[0.004716001616662413,
 0.018872827311055107,
 0.04514887879085716,
 0.09007912849955202,
 0.16487565786072922,
 0.15077014270281863,
 0.007186982044208645,
 0.07022991771243403,
 0.08721278502299246,
 0.2235616435078549,
 0.33074271563677804,
 0.15910129709384158,
 0.009568729428393041,
 0.039632090371146345,
 0.06607720934494532,
 0.11013112437867345,
 0.2733979187176665,
 0.3314712441896518,
 0.014925932527466546,
 0.10872124439812803,
 0.15251776138839468,
 0.12545499581063313,
 0.23110622740415,
 0.2491783456306269,
 0.020180732209634344,
 0.07139602906231057,
 0.1271503171413778,
 0.20702672501532604,
 0.2379405357685639,
 0.4469263359893184,
 0.04672998487796731,
 0.13889782860516908,
 0.19065434925522884,
 0.2179510152621552,
 0.2264935935900939,
 0.47612895963403457,
 0.2472672809914348,
 0.3827873828724341,
 0.2131428377492981,
 0.29063657447233526,
 0.1471244010660584,
 0.5720549807536229]